In [9]:
from langchain_community.llms import Ollama

# model 
llm = Ollama(model="gemma2:latest")

# chain 실행
response = llm.invoke("지구의 자전 주기는?")
response

' 지구의 자전 주기는 약 24시간입니다. 이를 **하루**라고 합니다. 하지만, 지구 자전이 항상 정확하게 24시간으로 일어나는 것은 아닙니다. 달과 태양의 중력, 지구 표면의 불균형 등 여러 요인이 영향을 미쳐 자전 속도가 조금씩 변화합니다.\n\n따라서 우리가 매일 경험하는 하루는 실제로 "평균 용량 자전 주기"라고 불리는 약 23시간 56분으로 정의됩니다.\n\n\n'

# prompt template 이용해서 역할 부여

In [17]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# Ollama를 사용하여 Gemma2 모델 초기화
model = Ollama(model="gemma2:latest")

# ChatPromptTemplate 정의
prompt = ChatPromptTemplate.from_template(
    "당신은 금융 전문가 입니다. 질문에 친절하게 답을 해주세요. Let's start step by step. : {question}\n\n답변:"
)

# 체인 생성
chain = prompt | model | StrOutputParser()

# chain 호출
response = chain.invoke({"question": "금융 전문가가 하는 일과 직업 종류를 알려줘"})
print(response)

안녕하세요! 금융 전문가로서 여러분의 재정적인 성공을 도울 수 있어 기쁩니다. 

금융 전문가는 개인, 기업 및 정부의 재정 상황을 관리하고 조언하는 전문가입니다. 그들의 업무는 매우 다양하며 직종도 여러 가지가 있습니다. 몇 가지 주요 직군과 그들이 하는 일은 다음과 같습니다:

**1. 금융 자문사:**

* 개인이나 기업의 재정 목표를 파악하고, 투자 전략을 수립 및 관리합니다.
* 주식, 채권, 부동산 등 다양한 투자 상품에 대한 조언을 제공합니다.
* 은퇴 계획, 교육 자금 마련 등 장기적인 재정 계획을 지원합니다.

**2. 증권사:**

* 주식, 채권 등의 증권을 거래하고 매매를 조율합니다.
* 투자자들에게 증권 시장 정보 및 분석 자료를 제공합니다.
* 기업이 자금을 조달하기 위한IPO(초기 공개) 등을 지원합니다.

**3. 은행원:**

* 개인과 기업에게 예금, 대출 및 신용 카드 서비스를 제공합니다.
* 고객의 재정 상황을 분석하고 적합한 금융 상품을 추천합니다.
* 은행의 수익성 관리 및 위험 관리에 참여합니다.

**4. 회계사:**

* 기업의 재무 자료를 기록, 분석하고 보고서를 작성합니다.
* 세금 준비 및 관련 법규 컨설팅을 제공합니다.
* 기업의 경영 효율성을 개선하기 위한 조언을 제공합니다.

**5. 보험 설계사:**

* 다양한 종류의 보험 상품을 소개하고 고객에게 맞는 상품을 추천합니다.
* 보험 가입 절차를 지원하고 보험 약관에 대한 설명을 제공합니다.
* 고객의 사고나 질병 발생 시 보상금 청구 과정을 돕습니다.


이 외에도 금융 전문가로서 활동할 수 있는 직종은 매우 다양합니다.

무엇이 특히 관심 있으신지 알려주시면 더 자세한 정보를 제공해 드릴 수 있습니다!


# langchain, gemma2 이용해서 구글 뉴스 검색 후, 기능 제공
1. 뉴스 요약
2. 키워드 추출
3. 카테고리 분류
4. 뉴스에 대한 3가지 주요 point
5. 뉴스에 대한 인사이트까지 정리

In [23]:
# 라이브러리 설치
# !pip install googlesearch-python requests beautifulsoup4 langchain


# 뉴스 원문 가져오는게 안되어서 아직 구동 X 코드

In [22]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from googlesearch import search
import requests
from bs4 import BeautifulSoup

# Gemma 2 모델을 사용하는 Ollama 인스턴스 생성
llm = Ollama(model="gemma2")

# 뉴스 검색 함수
def search_news(query, num_results=5):
    """
    구글 뉴스에서 주어진 쿼리로 뉴스를 검색합니다.
    :param query: 검색할 키워드
    :param num_results: 검색할 뉴스 개수
    :return: 검색된 뉴스 URL 리스트
    """
    search_results = search(query + " site:news.google.com", num_results=num_results)
    return list(search_results)

# 뉴스 내용 추출 함수
def extract_news_content(url):
    """
    주어진 URL에서 뉴스 내용을 추출합니다.
    :param url: 뉴스 URL
    :return: 추출된 뉴스 내용
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([p.text for p in paragraphs])
    return content[:1000]  # 첫 1000자만 반환

# 뉴스 요약 프롬프트 템플릿
summarize_template = """
다음 뉴스를 200자 이내로 요약해주세요:

{news_content}

요약:
"""
summarize_prompt = PromptTemplate(template=summarize_template, input_variables=["news_content"])
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

# 키워드 추출 및 카테고리 분류 프롬프트 템플릿
categorize_template = """
다음 뉴스 요약에서 주요 키워드 5개를 추출하고, 뉴스의 카테고리를 분류해주세요:

{news_summary}

형식:
키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5
카테고리: [경제, 주식, 부동산, 국제금융, 기업, 정책 중 하나 선택]
"""
categorize_prompt = PromptTemplate(template=categorize_template, input_variables=["news_summary"])
categorize_chain = LLMChain(llm=llm, prompt=categorize_prompt)

# 인사이트 도출 프롬프트 템플릿
insight_template = """
다음 뉴스 요약을 바탕으로 3가지 주요 포인트와 전반적인 인사이트를 제공해주세요:

{news_summary}

형식:
주요 포인트:
1. 
2. 
3. 

인사이트:
"""
insight_prompt = PromptTemplate(template=insight_template, input_variables=["news_summary"])
insight_chain = LLMChain(llm=llm, prompt=insight_prompt)

# 메인 함수
def analyze_financial_news(query):
    """
    금융 뉴스를 검색하고 분석하는 메인 함수입니다.
    :param query: 검색할 키워드
    """
    print(f"'{query}'에 대한 뉴스를 검색 중입니다...")
    news_urls = search_news(query)
    
    for url in news_urls:
        print(f"\n뉴스 URL: {url}")
        
        # 뉴스 내용 추출
        content = extract_news_content(url)
        
        # 뉴스 요약
        summary = summarize_chain.run(content)
        print(f"\n요약:\n{summary}")
        
        # 키워드 추출 및 카테고리 분류
        categorization = categorize_chain.run(summary)
        print(f"\n키워드 및 카테고리:\n{categorization}")
        
        # 인사이트 도출
        insights = insight_chain.run(summary)
        print(f"\n인사이트:\n{insights}")
        
        print("\n" + "="*50 + "\n")

# 사용 예시
if __name__ == "__main__":
    analyze_financial_news("한국 금융시장 동향")

'한국 금융시장 동향'에 대한 뉴스를 검색 중입니다...

뉴스 URL: https://news.google.com/__i/rss/rd/articles/CBMiM2h0dHBzOi8va3IuaW52ZXN0aW5nLmNvbS9lcXVpdGllcy90b3Atc3RvY2stZ2FpbmVyc9IBAA?oc=5

요약:
##  뉴스 요약을 위해 뉴스 내용을 입력해 주세요! 😊 
 
 뉴스 내용을 알려주시면, 200자 이내로 정리해서 드릴게요! 📰✨  


키워드 및 카테고리:
알겠습니다! 뉴스 내용을 입력해주세요. 😊  

내용을 알려주시면 200자 이내로 요약하고, 주요 키워드 5개와 카테고리를 분류해 드리겠습니다.  📰✨ 




인사이트:
뉴스 요약을 입력해 주세요! 😊  제가 멋진 요약과 인사이트를 드리도록 기대됩니다! ✨📰  


예시: "국내 최초로 AI 작곡 프로그램 개발..." 와 같이 뉴스 내용을 알려주세요. 😄  



뉴스 URL: https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3LmFzaWF0aW1lLmNvLmtyL2FydGljbGUvMjAyMzAzMDk1MDAwMjTSAQA?oc=5

요약:
제공된 뉴스가 없습니다. 뉴스를 입력해 주시면 200자 이내로 요약해 드리겠습니다! 😊  


키워드 및 카테고리:
알겠습니다! 뉴스를 입력해주시면  200자 이내로 요약해 드리고, 키워드 5개와 카테고리를 분류해 드리겠습니다. 😊  


기다리고 있어요! 👍  


인사이트:
뉴스를 입력해주세요!  📰✨ 

제가 뉴스를 읽고 핵심 포인트와 흥미로운 통찰력을 분석해서 드릴게요. 😊   


**예시:**

"삼성전자, 신형 스마트폰 출시 예정...AI 기능 강화!"

그런 식으로 뉴스 주제를 알려주세요! 👍  




뉴스 URL: https://news.google.com/__i/rss/rd/articles/CBMiNmh0dHBzOi8vd3d3Lmhhbmt5dW5nLmNvbS9lY29ub215L2FydGl

# 다른 방법으로 시도 

In [24]:
# 라이브러리 설치
# !pip install -q langchain-openai google-search-results